In [1]:
path = 'C:/Users/ok/Documents/House_data/House_data/Scripts/'

In [2]:
import pandas as pd
import sys
sys.path.append(path)
import Data_cleaner
cleaner = Data_cleaner.data_cleaner()

In [3]:
filepath = 'C:/Users/ok/Documents/House_data/House_data/Data/train.csv'
data = pd.read_csv(filepath)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [4]:
percent = cleaner.percent_missing(data)

In [5]:
print(f"{percent}%")

6.62%


devide the data into catagorical and numerical catagories
calculate the sum of null in both catagories
calculate the skewness of numerical data
fill the missing value for both catagories

# Fill in NaN values
Before we impute the NaN values, let's first classify the data into numerical and catagorical data.
This would help us to determine what method we will use to impute the data.
## Classifying the data into Numerical and Catagorical data depending on the type of the data.

In [6]:
catagorical = cleaner.get_categorical_columns(data)
numerical = cleaner.get_numerical_columns(data)

In [7]:
numerical

['Id',
 'MSSubClass',
 'LotFrontage',
 'LotArea',
 'OverallQual',
 'OverallCond',
 'YearBuilt',
 'YearRemodAdd',
 'MasVnrArea',
 'BsmtFinSF1',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 '1stFlrSF',
 '2ndFlrSF',
 'LowQualFinSF',
 'GrLivArea',
 'BsmtFullBath',
 'BsmtHalfBath',
 'FullBath',
 'HalfBath',
 'BedroomAbvGr',
 'KitchenAbvGr',
 'TotRmsAbvGrd',
 'Fireplaces',
 'GarageYrBlt',
 'GarageCars',
 'GarageArea',
 'WoodDeckSF',
 'OpenPorchSF',
 'EnclosedPorch',
 '3SsnPorch',
 'ScreenPorch',
 'PoolArea',
 'MiscVal',
 'MoSold',
 'YrSold',
 'SalePrice']

In [8]:
cat = data[catagorical]
num = data[numerical]

In [9]:
cat.isnull().sum()

MSZoning            0
Street              0
Alley            1369
LotShape            0
LandContour         0
Utilities           0
LotConfig           0
LandSlope           0
Neighborhood        0
Condition1          0
Condition2          0
BldgType            0
HouseStyle          0
RoofStyle           0
RoofMatl            0
Exterior1st         0
Exterior2nd         0
MasVnrType        872
ExterQual           0
ExterCond           0
Foundation          0
BsmtQual           37
BsmtCond           37
BsmtExposure       38
BsmtFinType1       37
BsmtFinType2       38
Heating             0
HeatingQC           0
CentralAir          0
Electrical          1
KitchenQual         0
Functional          0
FireplaceQu       690
GarageType         81
GarageFinish       81
GarageQual         81
GarageCond         81
PavedDrive          0
PoolQC           1453
Fence            1179
MiscFeature      1406
SaleType            0
SaleCondition       0
dtype: int64

In [10]:
num.isnull().sum()

Id                 0
MSSubClass         0
LotFrontage      259
LotArea            0
OverallQual        0
OverallCond        0
YearBuilt          0
YearRemodAdd       0
MasVnrArea         8
BsmtFinSF1         0
BsmtFinSF2         0
BsmtUnfSF          0
TotalBsmtSF        0
1stFlrSF           0
2ndFlrSF           0
LowQualFinSF       0
GrLivArea          0
BsmtFullBath       0
BsmtHalfBath       0
FullBath           0
HalfBath           0
BedroomAbvGr       0
KitchenAbvGr       0
TotRmsAbvGrd       0
Fireplaces         0
GarageYrBlt       81
GarageCars         0
GarageArea         0
WoodDeckSF         0
OpenPorchSF        0
EnclosedPorch      0
3SsnPorch          0
ScreenPorch        0
PoolArea           0
MiscVal            0
MoSold             0
YrSold             0
SalePrice          0
dtype: int64

### We can see that there are 18 columons (16 from catagorical columns and 2 from numerical columns).
### But from the data description, we can see that most of the NaN values were left empty because the house does  not have that feature.
* For instance, if the house does not have a Garage, the  garage type, quality, finish, condition, and year built will all be NaN values. Hence, we must be careful not to impute this type of columns. Because it would affect the prediction in the future. 
* INstead, for these columns we need to put a value( 0 in this case) to show that NaN is infact a value itself.
* FOr other columns such as Electricla instalation and Lot Frontage, we will use the appropriate method of imputation.

In [14]:
cols = ('Alley', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'FireplaceQu', 'GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageQual', 'GarageCond', 'PoolQC', 'Fence', 'MiscFeature', 'MasVnrType')

In [15]:
for col in cols:
    data[col] = data[col].fillna(0)
    return data

SyntaxError: 'return' outside function (3508064651.py, line 3)

In [1]:
data[cols] = (col for col in cols data[col]= data[col].fillna(0))

SyntaxError: invalid syntax (943562808.py, line 1)

In [12]:
import importlib

importlib.reload(Data_cleaner)

<module 'Data_cleaner' from 'C:\\Users/ok/Documents/House_data/House_data/Scripts\\Data_cleaner.py'>

In [13]:
cleaner.Nan_to_zero(data)

KeyError: ('Alley', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'FireplaceQu', 'GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageQual', 'GarageCond', 'PoolQC', 'Fence', 'MiscFeature', 'MasVnrType')